## 1. Import packages

In [1]:
#!pip install tweepy

In [2]:
import os
import tweepy as tw
import pandas as pd
from tqdm import tqdm

## 2. Twitter API authentication

In [3]:
consumer_api_key = os.environ["TWITTER_CONSUMER_API_KEY"]
consumer_api_secret = os.environ["TWITTER_CONSUMER_API_SECRET"]

In [4]:
auth = tw.OAuthHandler(consumer_api_key, consumer_api_secret)

In [5]:
api = tw.API(auth, wait_on_rate_limit=True)

## 3. Tweets query

### 3.1. Define the query

In [6]:
search_words = "#covid19 -filter:retweets"
date_since = "2020-03-01"
# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(7500)

### 3.2. Retreive the tweets

In [7]:
tweets_copy = []
for tweet in tqdm(tweets):
    tweets_copy.append(tweet)

1060it [00:32, 32.88it/s]


In [8]:
print(f"new tweets retrieved: {len(tweets_copy)}")

new tweets retrieved: 1060


## 4. Populate the dataset

In [9]:
tweets_df = pd.DataFrame()
for tweet in tqdm(tweets_copy):
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_created': tweet.user.created_at,
                                               'user_followers': tweet.user.followers_count,
                                               'user_friends': tweet.user.friends_count,
                                               'user_favourites': tweet.user.favourites_count,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': tweet.text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source,
                                               'is_retweet': tweet.retweeted}, index=[0]))

100%|██████████| 1060/1060 [00:04<00:00, 247.04it/s]


In [10]:
tweets_df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,A K KAUSHAL,ashokaushal@gmail.com,"Cost Accountant,CCA Rules RTI& Sexual Harassme...",2013-09-06 08:24:36,410,3115,2507,False,2020-08-07 13:08:52,"#20LakhCoronaPackage \nTests have reached 2,27...",[20LakhCoronaPackage],Twitter Web App,False
0,Andrea D. Furlan MD PhD,"Toronto, Ontario, Canada",I'm a physician trying to improve the person t...,2009-08-01 14:29:02,3847,1732,11053,False,2020-08-07 13:08:49,Vitamin D: A Low-Hanging Fruit in COVID-19? ht...,"[VitaminD, Covid19]",Twitter Web App,False
0,Superlexify,Wiradjuri country,"Treaty, gender equality, LGBTIQ rights, disabi...",2013-08-20 04:11:12,363,1444,25176,False,2020-08-07 13:08:48,"#NSW residents, time to show some spirit. No m...",[NSW],Twitter for iPhone,False
0,'Women's Power' Book,UK,Inspired by JohnGreen's #book topping NewYork ...,2011-07-03 19:08:35,29786,19509,207,False,2020-08-07 13:08:47,@MaximeBernier Wrong: #covid19 GLOBALLY spread...,"[covid19, lifeonearth, Foodie]",Twitter Web App,False
0,The Hon. Tony Clement,Muskoka Ontario,🇨🇦 Tech Entrepreneur; International Advisor; C...,2010-03-09 16:56:20,72663,55161,2797,True,2020-08-07 13:08:47,This Sunday on @AAThingPodcast we interview Dr...,None,Twitter for iPhone,False


## 5. Save the data

### 5.1. Read past data

In [11]:
tweets_old_df = pd.read_csv("covid19_tweets.csv")
print(f"past tweets: {tweets_old_df.shape}")

past tweets: (81650, 13)


### 5.2. Merge past and present data

In [12]:
tweets_all_df = pd.concat([tweets_old_df, tweets_df], axis=0)
print(f"new tweets: {tweets_df.shape[0]} past tweets: {tweets_old_df.shape[0]} all tweets: {tweets_all_df.shape[0]}")

new tweets: 1060 past tweets: 81650 all tweets: 82710


### 5.3. Drop duplicates

In [13]:
tweets_all_df.drop_duplicates(subset = ["user_name", "date", "text"], inplace=True)
print(f"all tweets: {tweets_all_df.shape}")

all tweets: (82710, 13)


### 5.4. Export the updated data

In [14]:
tweets_all_df.to_csv("covid19_tweets.csv", index=False)